In [22]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import numpy as np
from tqdm import tqdm

import sys 
sys.path.append('..')

from data_yours import FORAGE, ALS_50K
from model_yours import MAE3D, PatchEmbed_DGCNN

from pathlib import Path
import pandas as pd 

In [4]:
# get hydra config 
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from omegaconf import OmegaConf

if GlobalHydra.instance().is_initialized():
	GlobalHydra.instance().clear()
	
initialize(config_path="../config/experiments")
cfg = compose(config_name="/BASELINE_FORSPECIES") # SET CONFIG HERE
OmegaConf.set_struct(cfg, False)

/tmp/ipykernel_123966/3061211914.py:9: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="../config/experiments")


In [12]:
device = torch.device('cpu')

In [16]:
dgcnn = PatchEmbed_DGCNN(
    k=20, 
    output_channels=512
).cuda()

In [19]:
tensor = torch.rand(1, 3, 1028).cuda()

In [20]:
dgcnn(tensor)

tensor([[[ 1.3610e-01,  8.6316e-01, -1.0301e-01,  ...,  2.4199e+00,
          -3.5060e-01, -3.6402e-01],
         [ 8.9299e-01,  6.0541e-01,  1.8789e-03,  ...,  1.5062e+00,
           6.4045e-01, -1.6186e-01],
         [ 2.9796e+00, -7.3184e-02, -1.1685e-01,  ...,  2.4554e+00,
           2.5535e-01, -3.4236e-01],
         ...,
         [-2.1923e-01,  3.0160e-01,  1.0291e+00,  ...,  2.1515e-01,
          -5.8812e-01, -4.0880e-01],
         [-4.1450e-02,  1.0064e+00,  8.0467e-02,  ..., -1.5283e-01,
          -2.8948e-01, -2.6712e-01],
         [ 3.0247e+00,  5.5763e-01, -9.0539e-02,  ...,  1.8869e+00,
           1.1257e+00, -3.7108e-02]]], device='cuda:0',
       grad_fn=<LeakyReluBackward0>)

## Implementation of paper 
https://www.researchgate.net/profile/Gunho-Sohn/publication/277383984_Tree_genera_classification_with_geometric_features_from_high-density_airborne_LiDAR/links/56356c7808aeb786b702c58e/Tree-genera-classification-with-geometric-features-from-high-density-airborne-LiDAR.pdf


In [23]:
ds = ALS_50K()

In [24]:
tree = ds[0][0]

In [26]:
tree.shape

(2048, 3)

In [53]:
from sklearn.cluster import KMeans, DBSCAN

In [30]:
def initial_kmeans(points, k=100):
    # points: (N, 3) numpy array or tensor
    if isinstance(points, torch.Tensor):
        points_np = points.cpu().numpy()
    else:
        points_np = points
    kmeans = KMeans(n_clusters=k).fit(points_np)
    labels = kmeans.labels_
    return labels

In [54]:
def intial_dbscan(points, eps=0.2, min_samples=5):
    # points: (N, 3) numpy array or tensor
    if isinstance(points, torch.Tensor):
        points_np = points.cpu().numpy()
    else:
        points_np = points
    dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(points_np)
    labels = dbscan.labels_
    return labels

In [58]:
clusters_dbscan = intial_dbscan(tree, eps=0.01, min_samples=5)
tree_with_dbscan = np.concatenate([tree, clusters_dbscan[:, None]], axis=1)
np.savetxt('tree_with_dbscan.txt', tree_with_dbscan)

In [35]:
clusters = initial_kmeans(tree, k=100)
tree_with_clusters = np.concatenate([tree, clusters[:, None]], axis=1)

In [39]:
np.savetxt('inital_clusters.txt', tree_with_clusters, delimiter=',', fmt='%.6f')

In [48]:
clusters = [tree_with_clusters[tree_with_clusters[:, -1] == i, :-1] for i in range(100)]

In [49]:
centroids = [np.mean(cluster, axis=0) for cluster in clusters]

In [51]:
from scipy.spatial.distance import cdist

# Calculate pairwise distances between all centroids
centroids_array = np.stack(centroids)  # shape (100, 3)
pairwise_distances = cdist(centroids_array, centroids_array)  # shape (100, 100)
pairwise_distances.shape

(100, 100)